In [28]:
import argparse
# apply admm matrix factorization -> how to get the first order easily?
from src.preprocess.ml100k import ML100k
from src.models.mf import MF
from src.utils.trainer import Trainer
import torch

argparser = argparse.ArgumentParser()
argparser.add_argument('--latent_dim', type=int, default=10)
argparser.add_argument('--dropout', type=float, default=0.1)
argparser.add_argument('--batch_size', type=int, default=500)
argparser.add_argument('--epochs', type=int, default=100)
argparser.add_argument('--lr', type=float, default=0.01)
argparser.add_argument('--weight_decay', type=float, default=0.1)
args = argparser.parse_args("")
ml100k=ML100k(args)
data=ml100k.get_dataframe()
#mf_model=MF(args,ml100k.num_users,ml100k.num_items)

In [60]:
# train_test_split
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2, random_state=42)
import numpy as np

# train the model with Alternating Least Squares

# train the model with Alternating Least Squares
user_embedding=np.random.rand(ml100k.num_users,args.latent_dim)
item_embedding=np.random.rand(ml100k.num_items,args.latent_dim)
MAXITER=20
rating_matrix=data.pivot(index='user',columns='item',values='rating').values
rating_matrix=np.nan_to_num(rating_matrix)
# to integer
rating_matrix=rating_matrix.astype(int)
for iter in range(MAXITER):

    #update user embedding
    for i,ri in enumerate(rating_matrix):
        # user_id=train.iloc[i,0]
        # item_id=train.iloc[i,1]
        # rating=train.iloc[i,2]
        # ie=item_embedding[item_id]
        
        user_embedding[i]=np.linalg.solve(np.dot(item_embedding.T,np.dot(np.diag(ri),item_embedding))+args.weight_decay*np.eye(args.latent_dim),
                                          np.dot(item_embedding.T,np.dot(np.diag(ri),ri.T))).T

    #update item embedding

    for j,rj in enumerate(rating_matrix.T):
        # user_id=train.iloc[i,0]
        item_embedding[j]=np.linalg.solve(np.dot(user_embedding.T,np.dot(np.diag(rj),user_embedding))+args.weight_decay*np.eye(args.latent_dim),
                                            np.dot(user_embedding.T,np.dot(np.diag(rj),rj)))
        #item_embedding[item_id]=np.linalg.solve(np.dot(ue.T,ue)+np.eye(args.weight_decay),np.dot(ue.T,rating))
    
    # printmse
    print("iter: ",iter)
    loss=0
    count=0 
    for i in range(len(rating_matrix)):
        for j in range(len(rating_matrix.T)):
            if rating_matrix[i,j]>0:
                loss+=(np.dot(user_embedding[i],item_embedding[j])-rating_matrix[i,j])**2
                count+=1
    print("loss: ",loss/count)

print("done")

iter:  0
loss:  0.7747212612725982
iter:  1
loss:  0.6734309042478926
iter:  2
loss:  0.6301114095668406
iter:  3
loss:  0.6043279322024999
iter:  4
loss:  0.5876304894593881
iter:  5
loss:  0.5764152810970367
iter:  6
loss:  0.5684765844670439
iter:  7
loss:  0.5625272785664217
iter:  8
loss:  0.5578898492942498
iter:  9
loss:  0.5541091176559931
iter:  10
loss:  0.5509647569966503
iter:  11
loss:  0.5482608984944266
iter:  12
loss:  0.5458717338348794
iter:  13
loss:  0.5437248910840397
iter:  14
loss:  0.541789653639585
iter:  15
loss:  0.54006162735825
iter:  16
loss:  0.5385328768173466
iter:  17
loss:  0.5371761271115967
iter:  18
loss:  0.5359710590640292
iter:  19
loss:  0.5349022434297026
done
